In [1]:
import syft as sy

In [2]:
from syft.core.store.storeable_object import StorableObject

In [3]:
from syft.core.common import UID

In [4]:
import torch

# test functions

In [5]:
from syft.core.common.serde.serialize import _serialize
from syft.core.common.serde.deserialize import _deserialize
def serde(data, via_proto=True):
    to_proto = True if via_proto else False
    to_bytes = not to_proto
    from_proto = to_proto
    from_bytes = to_bytes
    
    data_serde = _deserialize(
        blob=_serialize(
            obj=data,
            to_proto=to_proto,
            to_bytes=to_bytes
        ),
        from_proto=from_proto,
        from_bytes=from_bytes
    )
    
    return data_serde

In [6]:
def test_serde(data, via_proto=True):
    data_serde = serde(data, via_proto=via_proto)
    assert data_serde==data

In [7]:
def test_StorableObject_data(data, via_proto=True):
    storable = StorableObject(id=UID(), data=data)
    storable_serde = serde(storable, via_proto=via_proto)
    assert storable_serde.data==data

In [8]:
remote = sy.VirtualMachine().get_root_client()
def test_send_get(data, remote=remote):
    tags = ["tag#"]
    description = "test description"
    ptr = data.send(remote, tags=tags, description=description, searchable=True)
    assert remote.store[-1].tags==ptr.tags==tags
    assert remote.store[-1].description==ptr.description==description
    assert remote.store[-1].id_at_location==ptr.id_at_location
    assert ptr.get()==data

In [9]:
def test(data):
    test_serde(data, via_proto=True)
    test_serde(data, via_proto=False)
    
    test_StorableObject_data(data, via_proto=True)
    test_StorableObject_data(data, via_proto=False)
    
    test_send_get(data)

# Int

In [10]:
data = sy.lib.python.Int(77)
test(data)

# torch.device

In [11]:
data = torch.device('cpu')
test(data)

# String

In [12]:
data = sy.lib.python.String("Hello world")
test(data)

# Dict

In [13]:
data = sy.lib.python.Dict({
    "a": 1,
    "b": 2
})
test(data)